In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
class DVN(nn.Module):
    def __init__(self, n=5):
        super(DVN, self).__init__()
        in_sz = (n+1)*34
        self.ds1 = nn.Linear(in_sz, in_sz);
        self.bn1 = nn.BatchNorm1d(in_sz, eps=1)
        self.ds2 = nn.Linear(in_sz, in_sz*2);
        self.bn2 = nn.BatchNorm1d(in_sz*2, eps=1)
        self.ds3 = nn.Linear(in_sz*2, in_sz);
        self.bn3 = nn.BatchNorm1d(in_sz, eps=1)
        self.ds4 = nn.Linear(in_sz, 1, bias=False)
        
        nn.init.xavier_normal_(self.ds1.weight)
        nn.init.xavier_normal_(self.ds2.weight)
        nn.init.xavier_normal_(self.ds3.weight)
        nn.init.xavier_normal_(self.ds4.weight)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.ds1(x)))
        x = F.relu(self.bn2(self.ds2(x)))
        x = F.relu(self.bn3(self.ds3(x)))
        return self.ds4(x)

In [36]:
# state: players (N) + merged (1) x token (1) + card (33)  

N = 5

def mutate(state, pot, action):
    state, pot = state.copy(), pot.copy()
    if state[0,0] == 0:
        action = False
    if action:
        state[[0,-1],0] -= 1
        pot[0] += 1
    else:
        state[0] += pot
        state[-1] += pot
        pot *= 0
    return state, pot
        
def next_turn(state):
    return np.r_[state[1:-1], state[0:1], state[-1:]]

def to_tensor(state):
    return torch.from_numpy(state.reshape((1, -1))).float().to(device)

def evaluate(net, state):
    net.eval()
    return net(to_tensor(state)).data.cpu().numpy()[0,0]

def score(state):
    out = state[0,0]
    if state[0,1] == 1: 
        out -= 3
    for i in range(2, 34):
        if state[0,i] == 1 and state[0,i-1] == 0:
            out -= i + 2
    return out
    
def spawn_player(value_net):
    def player(state, pot):
        if state[0,0] == 0: 
            return False
        sT = mutate(state, pot, True)[0]
        sF = mutate(state, pot, False)[0]
        vT = evaluate(value_net, sT)
        vF = evaluate(value_net, sF)
        return vT > vF
    return player

In [37]:
vns = [DVN(N).to(device) for _ in range(N)]
players = [spawn_player(vn) for vn in vns]

In [48]:
def gym(players):
    card = np.arange(33) + 3
    
    deck = card.copy()
    np.random.shuffle(deck)
    deck = list(deck[:24])
    def draw():
        return np.r_[0, (card.copy() == deck.pop()).astype(int)]

    n = len(players)
    state = np.r_[[np.r_[11, np.zeros((33,), int)] for _ in range(n)]]
    state = np.r_[state, state.sum(axis=0).reshape((1, -1))]
    
    turn = np.random.randint(n)
    replay = []
    
    while len(deck) > 0:
        pot = draw()
        while True:
            nothanks = players[turn](state, pot)
            state_, pot_ = mutate(state, pot, nothanks)
            reward = score(state_) - score(state)
            replay.append((state, state_, reward, turn))
            if nothanks:
                turn = (turn + 1)%n
                state = next_turn(state_)
                pot = pot_
            else:
                state = state_
                break
                
#     print("game end:")
#     for _ in range(n):
#         print(turn, ":", score(state))
#         turn = (turn + 1)%n
#         state = next_turn(state)
        
#     print()
    
#     print(state)
    
#     print()
            
#     for s, s_, r, t in replay:
#         print("turn:", t)
#         print(s, score(s))
#         print(s_, score(s_))
#         print("reward:", r)
        
    return replay

replay = gym(players)

In [49]:
replay

[(array([[11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0],
         [11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0],
         [11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0],
         [11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0],
         [11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0],
         [55,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0]